In [1]:
import pandas as pd
from fourinarowfunctions import *

In [2]:
datadir = '../data/'
filename = datadir + 'trialdata.csv'
data_dict = load_data(filename, verbose = False)
parsed_dict = {}
for username, data in data_dict.items():
    try:
        parsed_dict[username] = get_parsed_data(data, username)
    except AssertionError as e:
        continue
usernames = sorted(parsed_dict.keys())
usernames.remove("A1153:R_1CwnNA5xXOlHq5l")
usernames.remove("866:R_31YEgt5MqfcfZMC")
usernames.remove("A242:R_ymscU32j53VKwil")
print(f"{len(usernames)} users with complete data")

user A242:R_ymscU32j53VKwil started games after completing the task. Using only the first 37 games
156 users with complete data


In [3]:
quiz_list = get_quiz_answers(data_dict)
quiz_df = pd.DataFrame(quiz_list)
quiz_df = quiz_df[quiz_df["subject"].isin(usernames)]
quiz_df["count"] = 1
quiz_df.head()

,subject,image,expected,given,correct,rt_ms,count
0,166:R_9Nac2hjg6u7BlS1,blackHorizontal.png,Black,Black,True,8770,1
1,166:R_9Nac2hjg6u7BlS1,whiteDiagonal.png,White,White,True,2511,1
2,166:R_9Nac2hjg6u7BlS1,whiteVertical.png,White,White,True,2132,1
3,166:R_9Nac2hjg6u7BlS1,whiteNoWin.png,No-one,No-one,True,6503,1
4,166:R_9Nac2hjg6u7BlS1,blackDiagonal.png,Black,Black,True,1919,1


In [4]:
quiz_per_question = quiz_df.groupby("image").sum()
quiz_per_question["avg_rt_s"] = quiz_per_question["rt_ms"] / 1000 / quiz_per_question["count"]
quiz_per_question["correct_ratio"] = quiz_per_question["correct"] / quiz_per_question["count"]
quiz_per_question

,correct,rt_ms,count,avg_rt_s,correct_ratio
image,,,,,
blackDiagonal.png,155,391289,156,2.508263,0.993590
blackHorizontal.png,151,1040733,156,6.671365,0.967949
whiteDiagonal.png,155,411379,156,2.637045,0.993590
whiteNoWin.png,142,692258,156,4.437551,0.910256
whiteVertical.png,156,410528,156,2.631590,1.000000


In [5]:
quiz_per_subject = quiz_df.groupby("subject").sum()
quiz_per_subject.groupby("correct").nunique()

,rt_ms,count
correct,,
2,1,1
3,1,1
4,16,1
5,138,1
